<a href="https://colab.research.google.com/github/Vignesh-P-C/fake-news-detection-transformers/blob/main/06_final_training_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Training Run (Sanity Check)

**Project:** Fake News Detection using Transformers  
**Goal:** Verify end-to-end training works and loss decreases


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 256

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
fake_df = pd.read_csv(
    "Fake.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

true_df = pd.read_csv(
    "True.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

fake_df["label"] = 0
true_df["label"] = 1

df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# FORCE correct types
texts = df["text"].astype(str).tolist()
labels = df["label"].tolist()

In [ ]:
type(texts), type(texts[0])

(list, str)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [ ]:
assert isinstance(train_texts, list)
assert isinstance(train_texts[0], str)

In [ ]:
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


In [ ]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="no",
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.003294,0.002349


TrainOutput(global_step=282, training_loss=0.057345793338109415, metrics={'train_runtime': 53.8259, 'train_samples_per_second': 41.913, 'train_steps_per_second': 5.239, 'total_flos': 149423225683968.0, 'train_loss': 0.057345793338109415, 'epoch': 1.0})

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 0.002348913112655282,
 'eval_model_preparation_time': 0.0031,
 'eval_accuracy': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 4.0454,
 'eval_samples_per_second': 139.666,
 'eval_steps_per_second': 17.551}

In [ ]:
len(set(train_texts).intersection(set(val_texts)))

1

In [ ]:
len(val_texts)

565

In [ ]:
print("Train size:", len(train_texts))
print("Val size:", len(val_texts))

print("Overlap:", len(set(train_texts).intersection(set(val_texts))))


Train size: 2256
Val size: 565
Overlap: 1


In [ ]:
print(train_labels[:20])
print(val_labels[:20])

[0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0]


In [ ]:
predictions = trainer.predict(val_dataset)

logits = predictions.predictions
labels = predictions.label_ids

probs = torch.softmax(torch.tensor(logits), dim=1)
pred_classes = torch.argmax(probs, dim=1)

In [ ]:
confidences = probs.max(dim=1).values.numpy()

print("Min confidence:", confidences.min())
print("Mean confidence:", confidences.mean())
print("Max confidence:", confidences.max())

Min confidence: 0.6094278
Mean confidence: 0.9978374
Max confidence: 0.99867344


In [ ]:
import numpy as np

lowest_conf_indices = np.argsort(confidences)[:5]

for idx in lowest_conf_indices:
    print("Confidence:", confidences[idx])
    print("True label:", labels[idx])
    print("Predicted:", pred_classes[idx].item())
    print("Text snippet:", val_texts[idx][:200])
    print("-" * 50)

Confidence: 0.6094278
True label: 0
Predicted: 0
Text snippet: On Tuesday, Education Secretary Betsy Devos announced that she will appoint A. Wayne Johnson as the as Chief Operating Officer of Federal Student Aid on Thursday. In the press release put out by the E
--------------------------------------------------
Confidence: 0.9970644
True label: 0
Predicted: 0
Text snippet: This week s cold open for Saturday Night Live featured a parody of the morning show, Morning Joe. The sketch featured Alex Moffat as host Joe Scarborough and Kate McKinnon as Mika Mika Brzezinski. The
--------------------------------------------------
Confidence: 0.9971168
True label: 1
Predicted: 1
Text snippet: The following statements were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own. Reuters has not edited the state
--------------------------------------------------
Confidence: 0.99735934
True label: 1
Predicted: 1
Text 

In [ ]:
num_train_epochs=3,

In [ ]:
save_strategy="epoch",

In [ ]:
load_best_model_at_end=True,
metric_for_best_model="f1",
greater_is_better=True,

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000036,0.011407,0.998230,0.995918,1.000000,0.997955
2,0.000007,0.009895,0.998230,0.995918,1.000000,0.997955
3,0.000004,0.009924,0.998230,0.995918,1.000000,0.997955


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=846, training_loss=4.6774338720719736e-05, metrics={'train_runtime': 194.987, 'train_samples_per_second': 34.71, 'train_steps_per_second': 4.339, 'total_flos': 448269677051904.0, 'train_loss': 4.6774338720719736e-05, 'epoch': 3.0})

In [ ]:
trainer.save_model("final_model")
tokenizer.save_pretrained("final_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('final_model/tokenizer_config.json', 'final_model/tokenizer.json')

In [ ]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer

loaded_model = DistilBertForSequenceClassification.from_pretrained("final_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("final_model")

print("Model loaded successfully.")

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

Model loaded successfully.


In [ ]:
import torch

def predict_news(text):
    inputs = loaded_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = loaded_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()

    label = "REAL" if pred == 1 else "FAKE"
    confidence = probs[0][pred].item()

    return {
        "label": label,
        "confidence": confidence
    }

In [ ]:
predict_news("The president announced new economic reforms today.")

{'label': 'REAL', 'confidence': 0.9765084385871887}